### **Lab 3 - Parte 2** 

In [1]:
#import opendatasets as od # no para python 3.13
import pandas as pd
import os
import zipfile
import matplotlib.pyplot as plt
from matplotlib.image import imread
import random

from os import listdir
import shutil
import numpy as np
import keras.preprocessing.image as kerasImg
import keras.layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import ops



### **Entrenamiento Modelo Red neuronal y SVM**

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt

img_height = 28
img_width = 28
batch_size = 64
data_dir = 'D:/Documentos/Octavo Semestre/Data/lab3_datascience/PolyMNIST/MMNIST/train'


In [ ]:
def normalizar(imagen, etiqueta):
    imagen = tf.cast(imagen, tf.float32) / 255.0
    return imagen, etiqueta

train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size
).map(normalizar)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size
).map(normalizar)

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

num_classes = len(train_dataset.element_spec[1].numpy().shape) if hasattr(train_dataset.element_spec[1], "numpy") else 5 


Found 300000 files belonging to 5 classes.
Using 240000 files for training.
Found 300000 files belonging to 5 classes.
Using 60000 files for validation.


In [5]:
from tensorflow.keras import layers, models, optimizers

model = models.Sequential([
    layers.Input(shape=(img_height, img_width, 1)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)


In [ ]:
loss, accuracy = model.evaluate(val_dataset)
print(f"Pérdida en validación: {loss:.4f}, Precisión: {accuracy:.4f}")


In [ ]:
# Graficar precisión y pérdida
plt.plot(history.history['accuracy'], label='Precisión entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión validación')
plt.legend()
plt.title("Precisión por época")
plt.grid()
plt.show()

plt.plot(history.history['loss'], label='Pérdida entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida validación')
plt.legend()
plt.title("Pérdida por época")
plt.grid()
plt.show()


In [1]:
import tensorflow as tf
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Parámetros
img_height = 28
img_width = 28
batch_size = 64
data_dir = 'D:/Documentos/Octavo Semestre/Data/lab3_datascience/PolyMNIST/MMNIST/train'

# Función para normalizar
def normalizar(img, label):
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

# Datasets de entrenamiento y validación
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size
).map(normalizar)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size
).map(normalizar)


c:\Users\USUARIO\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\USUARIO\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\USUARIO\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/f

Found 300000 files belonging to 5 classes.
Using 240000 files for training.
Found 300000 files belonging to 5 classes.
Using 60000 files for validation.


In [ ]:
def dataset_to_numpy(dataset):
    X, y = [], []
    for batch_images, batch_labels in dataset:
        X.append(batch_images.numpy())
        y.append(batch_labels.numpy())
    X = np.concatenate(X)
    y = np.concatenate(y)
    return X, y

X_train, y_train = dataset_to_numpy(train_dataset)
X_val, y_val = dataset_to_numpy(val_dataset)

# Aplanar imágenes para usar con SVM
X_train_flat = X_train.reshape((X_train.shape[0], -1))
X_val_flat = X_val.reshape((X_val.shape[0], -1))


In [ ]:
svm = SVC(kernel='rbf', C=10, gamma='scale')
svm.fit(X_train_flat, y_train)

# Predecir y evaluar
y_pred = svm.predict(X_val_flat)
acc = accuracy_score(y_val, y_pred)

print(f" Precisión del modelo SVM: {acc:.4f}")
print("\n Reporte de clasificación:\n")
print(classification_report(y_val, y_pred))


In [ ]:
# Probar otros kernels para comparar rendimiento
for kernel in ['linear', 'rbf', 'poly']:
    print(f"\n Entrenando SVM con kernel: {kernel}")
    model = SVC(kernel=kernel, C=1)
    model.fit(X_train_flat, y_train)
    preds = model.predict(X_val_flat)
    acc = accuracy_score(y_val, preds)
    print(f"Precisión: {acc:.4f}")
